<a href="https://colab.research.google.com/github/student64-ahmadi/Impulsive-Hamiltonian-Dynamics/blob/main/controle_actif_de%20coh%C3%A9rence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 71.4 MB/s eta 0:00:00


In [12]:
# -*- coding: utf-8 -*-
"""
Simulation Quantique Améliorée - Contrôle de la Décohérence (Corrigée v5.2)
Auteur: Ahmadi Othman (modifié par Grok 3)
Date: 2023-10-15 (mis à jour le 10 mars 2025)
"""

import numpy as np
import matplotlib.pyplot as plt
from qutip import sigmax, sigmaz, sigmam, qeye, tensor, basis, mesolve, entropy_vn, ket2dm, Options
import os
import zipfile as zf  # Alias pour éviter conflits
try:
    from google.colab import files
    colab_available = True
except ImportError:
    colab_available = False

# =============================================================================
# 1. Configuration globale
# =============================================================================
CONFIG = {
    "params": {
        "J": 1.0,                   # Couplage d'échange (GHz)
        "mu_values": [1.0],         # Test avec une seule valeur
        "gamma": 0.1,               # Taux de dissipation (GHz)
        "t_event": 5.0,             # Instant de l'impulsion unique (ns)
        "sigma": 0.01,              # Largeur gaussienne optimale (ns)
        "t_points": 200,            # Nombre de points temporels
        "initial_state": "bell",    # État initial : "bell" (|00⟩ + |11⟩)/√2
        "operators": ["sz_I"]       # Test avec un seul opérateur
    },
    "output": {
        "fig_dir": "figures_enhanced",
        "zip_file": "quantum_figures_enhanced.zip"
    }
}

CONFIG["params"]["tlist"] = np.linspace(0, 10, CONFIG["params"]["t_points"])

# =============================================================================
# 2. Définitions des fonctions principales
# =============================================================================
def setup_system(operator_type="sz_I"):
    """Initialise les opérateurs quantiques"""
    sx, sz, sm, I = sigmax(), sigmaz(), sigmam(), qeye(2)
    H0 = -CONFIG["params"]["J"] * tensor(sx, sx)
    H_imp = tensor(sz, I) if operator_type == "sz_I" else tensor(I, sz)
    c_ops = [
        np.sqrt(CONFIG["params"]["gamma"]) * tensor(sm, I),
        np.sqrt(CONFIG["params"]["gamma"]) * tensor(I, sm)
    ]
    return {"H0": H0, "H_imp": H_imp, "c_ops": c_ops}

def gaussian_pulse(t, args):
    """Profil temporel gaussien pour une impulsion unique"""
    mu = args["mu"]
    sigma = CONFIG["params"]["sigma"]
    t_event = CONFIG["params"]["t_event"]
    pulse = mu * np.exp(-((t - t_event)**2) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
    return pulse

def get_initial_state():
    """Retourne l'état initial"""
    if CONFIG["params"]["initial_state"] == "bell":
        return (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
    return tensor(basis(2, 0), basis(2, 0))

def run_simulation(mu, operator_type="sz_I"):
    """Exécute la simulation"""
    system = setup_system(operator_type)
    H = [system["H0"], [system["H_imp"], lambda t, args: gaussian_pulse(t, {"mu": mu})]]
    psi0 = get_initial_state()
    e_ops = [
        tensor(sigmaz(), qeye(2)),
        tensor(qeye(2), sigmaz()),
        tensor(basis(2, 0), basis(2, 0)) * tensor(basis(2, 0), basis(2, 0)).dag(),
        tensor(basis(2, 1), basis(2, 1)) * tensor(basis(2, 1), basis(2, 1)).dag()
    ]
    opts = Options(store_states=True)
    try:
        result = mesolve(H, psi0, CONFIG["params"]["tlist"], system["c_ops"], e_ops, options=opts)
        print(f"Simulation réussie pour mu={mu}, op={operator_type}: {len(result.states)} états, expect shape={np.shape(result.expect)}")
        return result
    except Exception as e:
        print(f"Erreur dans la simulation pour mu={mu}, op={operator_type}: {e}")
        return None

# =============================================================================
# 3. Fonctions de visualisation
# =============================================================================
def plot_density_matrix(results):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_density_matrix")
            continue
        rho_elements = [rho.ptrace(0).full() for rho in result.states]
        rho_00 = [np.real(rho[0,0]) for rho in rho_elements]
        rho_11 = [np.real(rho[1,1]) for rho in rho_elements]
        ax1.plot(CONFIG["params"]["tlist"], rho_00, label=f'{key}')
        ax2.plot(CONFIG["params"]["tlist"], rho_11, label=f'{key}')
        has_data = True
    ax1.set(ylabel='Re(ρ₀₀)', title='Évolution des éléments de la matrice densité (Qubit 1)')
    ax2.set(xlabel='Temps (ns)', ylabel='Re(ρ₁₁)')
    if has_data:
        ax1.legend()
        ax2.legend()
    plt.tight_layout()
    save_figures(fig, "density_matrix_evolution.pdf")

def plot_coherences(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_coherences")
            continue
        rho_elements = [rho.ptrace(0).full() for rho in result.states]
        coherences = [np.abs(rho[0,1]) for rho in rho_elements]
        ax.plot(CONFIG["params"]["tlist"], coherences, label=f'{key}')
        has_data = True
    ax.set(xlabel='Temps (ns)', ylabel='|ρ₀₁|', title='Évolution des cohérences (Qubit 1)')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "coherence_evolution.pdf")

def plot_entropy(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_entropy")
            continue
        entropies = [entropy_vn(rho.ptrace(0)) for rho in result.states]
        ax.plot(CONFIG["params"]["tlist"], entropies, label=f'{key}')
        has_data = True
    ax.set(xlabel='Temps (ns)', ylabel='Entropie (bits)', title='Entropie de von Neumann (Qubit 1)')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "entropy_evolution.pdf")

def plot_purity(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_purity")
            continue
        purities = [np.real((rho * rho).tr()) for rho in result.states]
        ax.plot(CONFIG["params"]["tlist"], purities, label=f'{key}')
        has_data = True
    ax.set(xlabel='Temps (ns)', ylabel='Tr(ρ²)', title='Évolution de la pureté quantique')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "purity_evolution.pdf")

def plot_population_transfer(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.expect:
            print(f"Données manquantes pour {key} dans plot_population_transfer")
            continue
        ax.plot(CONFIG["params"]["tlist"], result.expect[2], label=f'{key} |00⟩')
        ax.plot(CONFIG["params"]["tlist"], result.expect[3], linestyle='--', label=f'{key} |11⟩')
        has_data = True
    ax.set(xlabel='Temps (ns)', ylabel='Population', title='Transfert de population')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "population_transfer.pdf")

def plot_spectral_analysis(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_spectral_analysis")
            continue
        rho_elements = [rho.ptrace(0).full() for rho in result.states]
        coherences = [np.abs(rho[0,1]) for rho in rho_elements]
        freqs = np.fft.fftfreq(len(CONFIG["params"]["tlist"]), d=CONFIG["params"]["tlist"][1] - CONFIG["params"]["tlist"][0]) * 1e9
        fft_vals = np.abs(np.fft.fft(coherences))
        ax.plot(freqs[:len(freqs)//2], fft_vals[:len(freqs)//2], label=f'{key}')
        has_data = True
    ax.set(xlabel='Fréquence (Hz)', ylabel='Amplitude', title='Analyse spectrale des oscillations')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "spectral_analysis.pdf")

def plot_expectation_values(results):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    has_data = False
    for key, result in results.items():
        if result is None or not result.expect:
            print(f"Données manquantes pour {key} dans plot_expectation_values")
            continue
        ax1.plot(CONFIG["params"]["tlist"], result.expect[0], label=f'{key}')
        ax2.plot(CONFIG["params"]["tlist"], result.expect[1], label=f'{key}')
        has_data = True
    ax1.set(ylabel='<σz> Qubit 1', title='Valeurs moyennes de σz')
    ax2.set(xlabel='Temps (ns)', ylabel='<σz> Qubit 2')
    if has_data:
        ax1.legend()
        ax2.legend()
    plt.tight_layout()
    save_figures(fig, "expectation_values.pdf")

def plot_trace_validation(results):
    fig, ax = plt.subplots()
    has_data = False
    for key, result in results.items():
        if result is None or not result.states:
            print(f"Données manquantes pour {key} dans plot_trace_validation")
            continue
        traces = [np.real(rho.tr()) for rho in result.states]
        ax.plot(CONFIG["params"]["tlist"], traces, label=f'{key}')
        has_data = True
    ax.set(xlabel='Temps (ns)', ylabel='Tr(ρ)', title='Validation de la trace (devrait être 1)')
    if has_data:
        ax.legend()
    plt.tight_layout()
    save_figures(fig, "trace_validation.pdf")

def save_figures(fig, filename):
    """Sauvegarde les figures avec débogage"""
    try:
        os.makedirs(CONFIG["output"]["fig_dir"], exist_ok=True)
        filepath = os.path.join(CONFIG["output"]["fig_dir"], filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close(fig)
        if os.path.exists(filepath):
            print(f"Figure sauvegardée avec succès : {filepath}")
        else:
            print(f"Échec de la sauvegarde : {filepath} n'existe pas")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde de {filename} : {e}")

def create_zip_archive():
    """Crée une archive ZIP des figures"""
    try:
        zip_path = CONFIG["output"]["zip_file"]
        with zf.ZipFile(zip_path, 'w') as zipf:
            files_added = 0
            fig_dir = CONFIG["output"]["fig_dir"]
            for file in os.listdir(fig_dir):
                full_path = os.path.join(fig_dir, file)
                zipf.write(full_path, file)
                files_added += 1
            print(f"Archive créée : {zip_path} avec {files_added} fichiers")
        if colab_available:
            files.download(zip_path)
    except Exception as e:
        print(f"Erreur lors de la création de l'archive : {e}")

# =============================================================================
# 4. Exécution principale
# =============================================================================
if __name__ == "__main__":
    # Simulations
    results = {}
    for mu in CONFIG["params"]["mu_values"]:
        for op in CONFIG["params"]["operators"]:
            key = f"μ={mu}, op={op}"
            results[key] = run_simulation(mu, op)

    # Vérification des résultats
    if not results or all(r is None for r in results.values()):
        print("Aucun résultat valide dans results")
    else:
        print(f"Nombre de simulations réussies : {len([r for r in results.values() if r is not None])}")

    # Génération des figures
    plot_density_matrix(results)
    plot_coherences(results)
    plot_entropy(results)
    plot_purity(results)
    plot_population_transfer(results)
    plot_spectral_analysis(results)
    plot_expectation_values(results)
    plot_trace_validation(results)



/usr/local/lib/python3.11/dist-packages/qutip/solver/options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/qutip/solver/solver_base.py:576: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


Simulation réussie pour mu=1.0, op=sz_I: 200 états, expect shape=(4, 200)
Nombre de simulations réussies : 1
Figure sauvegardée avec succès : figures_enhanced/density_matrix_evolution.pdf
Figure sauvegardée avec succès : figures_enhanced/coherence_evolution.pdf
Figure sauvegardée avec succès : figures_enhanced/entropy_evolution.pdf
Figure sauvegardée avec succès : figures_enhanced/purity_evolution.pdf
Figure sauvegardée avec succès : figures_enhanced/population_transfer.pdf
Figure sauvegardée avec succès : figures_enhanced/spectral_analysis.pdf
Figure sauvegardée avec succès : figures_enhanced/expectation_values.pdf
Figure sauvegardée avec succès : figures_enhanced/trace_validation.pdf
